In [1]:
import json
from typing import TypedDict
import dotenv
import os

dotenv.load_dotenv()

False

In [44]:
class TestCaseFromFile(TypedDict):
    question: str
    ground_truth: str

class TestCaseFromFileFull(TypedDict):
    question: str
    ground_truth: str
    model_answer: str

type TestCasesFile = list[TestCaseFromFile]

type TestCasesFileFull = list[TestCaseFromFile]

class TestResult(TypedDict):
    score: float | None
    reason: str | None

class TestMetricsResults(TypedDict):
    correctness: TestResult
    clarity: TestResult

class TestOut(TypedDict):
    result: TestMetricsResults
    question: str
    model_answer: str
    ref_answer: str | None

class Out(TypedDict):
    summary: dict
    cases: list[TestOut]

## Getting model answers

In [3]:
address = "http://10.1.15.44:9006/v1/chat/completions"

In [4]:
import requests

def retrieve(text: str) -> str | None:
    messages = [
        {"role": "user", "content": text},
    ]

    payload = {
        "model": "menon-1",
        "messages": messages,
        "stream": False,
        "user": "test",
    }

    response = requests.post(address, json=payload)
    if response.status_code != 200:
        return None
    return response.json()["choices"][0]["message"]["content"]

In [5]:
import tqdm

with open("correct_json_full.json", "r") as f:
    jsoned_tests: TestCasesFile = json.load(f)

for jsoned_test in tqdm.tqdm(jsoned_tests):
    if "model_answer" not in jsoned_test or jsoned_test["model_answer"] is None:
        jsoned_test["model_answer"] = retrieve(jsoned_test["question"])

  0%|          | 0/75 [00:00<?, ?it/s]

100%|██████████| 75/75 [13:27<00:00, 10.76s/it]


### saving

In [6]:
jsoned_tests

[{'question': 'Здравствуйте! Как поступить на журналиста?',
  'ground_truth': 'Здравствуйте! Для поступления на направление "Журналистика. Современные медиатехнологии" в НГУ вам нужно выполнить несколько шагов. Вот подробное руководство, основанное на актуальной информации для абитуриентов 2025 года.\n\nКакие экзамены нужно сдать\n\nДля подачи заявления вам понадобятся результаты трех вступительных испытаний :\n\n1. Творческое испытание — профильный экзамен, который проводится непосредственно в университете.\n2. ЕГЭ по литературе.\n3. ЕГЭ по русскому языку.\n\nОбратите внимание: если вы поступаете на базе профессионального образования (СПО), вместо ЕГЭ по литературе сдать "Историю русской литературы".\n\nСроки подачи документов\n\nПодача документов для абитуриентов 2025 года на программы бакалавриата начинается 20 июня. Завершается прием: 25 июля — для тех, кто поступает по результатам ЕГЭ.\n\nПроцесс подачи заявления\n\nПодать документы можно одним из удобных способов:\n1. Лично в При

In [7]:
with open("correct_json_full.json", "w") as f:
    json.dump(jsoned_tests, f, indent=4, ensure_ascii=False)

## Judge model

### Google

#### Colab

In [ ]:
os.environ["GEMINI_API_KEY"] = ""

In [49]:
!uv pip install deepeval tqdm

Using Python 3.12.12 environment at: /usr
Audited 2 packages in 163ms


In [18]:
from google import genai

google_client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY"),
    http_options={"api_version": "v1beta"}
    # http_options={
    #     "httpx_client": proxied_http_client,
    #     "httpx_async_client": proxied_async_http_client,
    # },
)

In [19]:
(await google_client.aio.models.list()).page

[Model(
   description='Obtain a distributed representation of a text.',
   display_name='Embedding Gecko',
   input_token_limit=1024,
   name='models/embedding-gecko-001',
   output_token_limit=1,
   supported_actions=[
     'embedText',
     'countTextTokens',
   ],
   tuned_model_info=TunedModelInfo(),
   version='001'
 ),
 Model(
   description='Gemini 2.5 Pro Preview 03-25',
   display_name='Gemini 2.5 Pro Preview 03-25',
   input_token_limit=1048576,
   max_temperature=2.0,
   name='models/gemini-2.5-pro-preview-03-25',
   output_token_limit=65536,
   supported_actions=[
     'generateContent',
     'countTokens',
     'createCachedContent',
     'batchGenerateContent',
   ],
   temperature=1.0,
   thinking=True,
   top_k=64,
   top_p=0.95,
   tuned_model_info=TunedModelInfo(),
   version='2.5-preview-03-25'
 ),
 Model(
   description='Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.',
   display_nam

##### raw file to colab

In [38]:
encoded_string = """...
"""

##### Raw file unpack

In [41]:
!uv pip install zstd

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 72ms                                           
Prepared 1 package in 24ms                                               
Installed 1 package in 1ms                                  
 + zstd==1.5.7.2


In [45]:
import base64
import zstd

compressed_data = base64.b64decode(encoded_string)
json_string = zstd.decompress(compressed_data).decode('utf-8')
jsoned_data: TestCaseFromFileFull = json.loads(json_string)
jsoned_data

[{'question': 'Здравствуйте! Как поступить на журналиста?',
  'ground_truth': 'Здравствуйте! Для поступления на направление "Журналистика. Современные медиатехнологии" в НГУ вам нужно выполнить несколько шагов. Вот подробное руководство, основанное на актуальной информации для абитуриентов 2025 года.\n\nКакие экзамены нужно сдать\n\nДля подачи заявления вам понадобятся результаты трех вступительных испытаний :\n\n1. Творческое испытание — профильный экзамен, который проводится непосредственно в университете.\n2. ЕГЭ по литературе.\n3. ЕГЭ по русскому языку.\n\nОбратите внимание: если вы поступаете на базе профессионального образования (СПО), вместо ЕГЭ по литературе сдать "Историю русской литературы".\n\nСроки подачи документов\n\nПодача документов для абитуриентов 2025 года на программы бакалавриата начинается 20 июня. Завершается прием: 25 июля — для тех, кто поступает по результатам ЕГЭ.\n\nПроцесс подачи заявления\n\nПодать документы можно одним из удобных способов:\n1. Лично в При

### Local(proxy)

In [3]:
SOCKS5_PROXY_ADDRESS = "127.0.0.1:9150"

In [ ]:
import httpx

proxied_http_client = httpx.Client(proxy=f"socks5://{SOCKS5_PROXY_ADDRESS}")
proxied_async_http_client = httpx.AsyncClient(proxy=f"socks5://{SOCKS5_PROXY_ADDRESS}")

In [5]:
os.environ["wss_proxy"] = f"socks5://{SOCKS5_PROXY_ADDRESS}"
os.environ["ws_proxy"] = f"socks5://{SOCKS5_PROXY_ADDRESS}"

In [ ]:
from google import genai

google_client = genai.Client(
    api_key=os.getenv("GEMINI_API_KEY"),
    http_options={
        "httpx_client": proxied_http_client,
        "httpx_async_client": proxied_async_http_client,
    },
)

#### Model for deepeval

In [25]:
from deepeval.models import DeepEvalBaseLLM
import asyncio
import nest_asyncio


nest_asyncio.apply()


class NetworkModel(DeepEvalBaseLLM):

    def __init__(self, model_name: str, live: bool = False):
        self.model_name = model_name
        self.live = live
        if self.live:
            self.config = {
                "response_modalities": ["TEXT"],
            }
        else:
            self.config = {}

    def load_model(self):
        return self

    def generate(self, prompt: str) -> str:
        if self.live:
            return asyncio.run(self.a_generate(prompt))
        else:
            return google_client.models.generate_content(
                model=self.model_name,
                contents=prompt
            ).text
    
    async def a_generate(self, prompt: str) -> str:
        if self.live:
            async with google_client.aio.live.connect(model=self.model_name, config=self.config) as session:
                await session.send(input=prompt, end_of_turn=True)
                msgs = []
                async for msg in session.receive():
                    msgs.append(msg.text)
                return "".join(msg for msg in msgs if msg is not None)
        else:
            resp = await google_client.aio.models.generate_content(self.model_name, prompt)
            return resp.text

    def get_model_name(self):
        return self.model_name


In [46]:
judge_model = NetworkModel("gemini-live-2.5-flash-preview", True)
judge_model.generate("Что больше 9 или 5?")

/tmp/ipython-input-338608239.py:36: DeprecationWarning: The `session.send` method is deprecated and will be removed in a future version (not before Q3 2025).
Please use one of the more specific methods: `send_client_content`, `send_realtime_input`, or `send_tool_response` instead.
  await session.send(input=prompt, end_of_turn=True)


'9 больше, чем 5.'

# DeepEval

In [47]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams, LLMTestCase

correctness = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=judge_model
)
clarity = GEval(
    name="Clarity",
    evaluation_steps=[
        "Evaluate whether the response uses clear and direct language.",
        "Check if the explanation avoids jargon or explains it when used.",
        "Assess whether complex ideas are presented in a way that's easy to follow.",
        "Identify any vague or confusing parts that reduce understanding."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
    model=judge_model
)
# medical_faithfulness = GEval(
#     name="Medical Faithfulness",
#     evaluation_steps=[
#         "Extract medical claims or diagnoses from the actual output.",
#         "Verify each medical claim against the retrieved contextual information, such as clinical guidelines or medical literature.",
#         "Identify any contradictions or unsupported medical claims that could lead to misdiagnosis.",
#         "Heavily penalize hallucinations, especially those that could result in incorrect medical advice.",
#         "Provide reasons for the faithfulness score, emphasizing the importance of clinical accuracy and patient safety."
#     ],
#     evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.RETRIEVAL_CONTEXT],
# )

In [ ]:
out: Out = {
    "summary": {},
    "cases": [],
}

In [52]:
import tqdm

i = 0
for test_case in tqdm.tqdm(jsoned_data):
    if i < len(out["cases"]) and out["cases"][i]["question"] == test_case["question"]:
        i += 1
        continue
    i += 1
    llm_test_case = LLMTestCase(
        input=test_case["question"],
        actual_output=test_case["model_answer"],
        expected_output=test_case["ground_truth"]
    )
    
    correctness.measure(llm_test_case)
    print(correctness.score, correctness.reason)
    
    clarity.measure(llm_test_case)
    print(clarity.score, clarity.reason)
    out["cases"].append(
        TestOut(
            result=TestMetricsResults(
                correctness=TestResult(
                    score=correctness.score,
                    reason=correctness.reason
                ),
                clarity=TestResult(
                    score=clarity.score,
                    reason=clarity.reason
                ),
            ),
            question=test_case["question"],
            model_answer=test_case["model_answer"],
            ref_answer=test_case["ground_truth"]
        )
    )
print("Done with cases")
out

  0%|          | 0/75 [00:00<?, ?it/s]

Output()

/tmp/ipython-input-338608239.py:36: DeprecationWarning: The `session.send` method is deprecated and will be removed
in a future version (not before Q3 2025).
Please use one of the more specific methods: `send_client_content`, `send_realtime_input`, or `send_tool_response` 
instead.
  await session.send(input=prompt, end_of_turn=True)

Output()

0.2 The actual output heavily penalizes omission of detail, failing to provide the specific start and end dates for document submission, the different deadlines for internal exams vs. ЕГЭ, and the stages of enrollment, all of which are present in the expected output. While it doesn't directly contradict a fact, its vagueness and lack of specific information are a major shortcoming.


...

Output()

0.4 The actual output generally aligns with the expected output by suggesting checking the official university website or admissions committee. However, it fails to provide the crucial, specific link to the official admissions committee website (https://abiturient.nsu.ru/) mentioned in the expected output, which is a significant omission of detail. It also lacks the specific link for popular specialties, making it less helpful than the expected output.


100%|██████████| 75/75 [02:45<00:00,  2.21s/it]

0.9 The response uses clear and direct language, avoiding jargon effectively. Complex ideas, such as finding applicant ratings, are presented in an easy-to-follow manner by suggesting specific website sections and the admissions committee. There are no vague or confusing parts that reduce understanding.
Done with cases


{'summary': {},
 'cases': [{'result': {'correctness': {'score': 0.0,
     'reason': "The 'actual output' directly contradicts the 'expected output' by stating that NGU does not have a separate faculty or direction specifically for journalism, while the 'expected output' provides detailed information for admission to the 'Journalism. Modern Media Technologies' program at NGU. The actual output also vaguely suggests 'Faculty of Natural Sciences' for journalism-related programs, which is incorrect. Furthermore, there is a significant omission of crucial details regarding specific entrance exams (creative testing, Literature, Russian Language ЕГЭ), deadlines, document requirements, and a detailed description of the creative competition, all of which are present in the expected output."},
    'clarity': {'score': 0.9,
     'reason': 'The response uses clear and direct language, making it easy to understand for someone exploring journalism education. It avoids jargon and presents complex ide

In [53]:
out

{'summary': {},
 'cases': [{'result': {'correctness': {'score': 0.0,
     'reason': "The 'actual output' directly contradicts the 'expected output' by stating that NGU does not have a separate faculty or direction specifically for journalism, while the 'expected output' provides detailed information for admission to the 'Journalism. Modern Media Technologies' program at NGU. The actual output also vaguely suggests 'Faculty of Natural Sciences' for journalism-related programs, which is incorrect. Furthermore, there is a significant omission of crucial details regarding specific entrance exams (creative testing, Literature, Russian Language ЕГЭ), deadlines, document requirements, and a detailed description of the creative competition, all of which are present in the expected output."},
    'clarity': {'score': 0.9,
     'reason': 'The response uses clear and direct language, making it easy to understand for someone exploring journalism education. It avoids jargon and presents complex ide

In [63]:
clarity_avg = 0
correctness_avg = 0
for case in out["cases"]:
    clarity_avg += case["result"]["clarity"]["score"]
    correctness_avg += case["result"]["correctness"]["score"]
clarity_avg /= len(out["cases"])
correctness_avg /= len(out["cases"])
print(clarity_avg, correctness_avg)
out["summary"] = {
    "clarity": clarity_avg,
    "correctness": correctness_avg
}

0.8925531914893621 0.28723404255319135


In [55]:
with open("result.json", "w") as f:
    json.dump(out, f, indent=4, ensure_ascii=False)

In [64]:
out_str = json.dumps(out, indent=4, ensure_ascii=False)
base64.b64encode(zstd.encode(out_str.encode("utf-8"))).decode("utf-8")

'...'